In [1]:
import configparser
import datetime as dt
import sys
import collections
from pathlib import Path
from typing import Union, Optional, List, Tuple

import codebook.EDA as EDA
import codebook.clean as clean
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
from sqlalchemy import func, distinct

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.style.use('raph-base')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

np.random.seed(666)

In [87]:
sys.path.append(str(Path.cwd().parent))

from src.db_declaration import Base, Artist, CreditTrx, Genre, Label, Record, RecordFormat, RecordLabelLink
from src import db_functions
from src import db_connect

In [4]:
print(sys.executable)
print(sys.version)
print(f'sqlalchemy {sqlalchemy.__version__}')

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]
sqlalchemy 1.3.17


## Connect To And (Re-)Create DB

Because we connect from `dev`subfolder, I have to work with the sqlalchemy `create_engine` function directly.

In [31]:
db_params = {"REL_PATH": "DeafDiscoBase.db"}
engine = db_connect.create_engine(db_params)
session = db_connect.create_session(engine)

# Check
engine
session

Engine(sqlite:///C:\Users\r2d4\OneDrive\code\projects\20-02_disco\dev\DeafDiscoBase.db)

In [6]:
# db_functions.create_DB_anew(engine, Base)

In [7]:
def load_some_albums_from_xlsx(
    filepath: Union[Path, str],
    n: int=3,
    random_state: int=5,
) -> pd.DataFrame:
    """Load the original album collection file into a dataframe.
    You can specify a list of genres you want to include
    (defaults to None).
    """
    df = pd.read_excel(filepath, engine="openpyxl")
    return df.sample(n=n, random_state=random_state)


def clean_collection(collection_df):
    collection_df = clean.prettify_column_names(collection_df)
    collection_df.dropna(thresh=8, inplace=True)
    collection_df.rename(columns={"format": "record_format"}, inplace=True)
    collection_df["label"].fillna("NA", inplace=True)  # because it creates a foreign key
    collection_df["vinyl_color"] = np.nan  # just for aesthetics ...
    collection_df["artist_country"] = None
    collection_df["credit_value"] = 0
    collection_df["trx_type"] = "Initial Load"
    return collection_df

In [9]:
CONFIG_PATH = Path.cwd().parent / "config.yaml"
path_to_collection = db_connect.read_yaml(CONFIG_PATH, "COLLECTION")
collection_df = load_some_albums_from_xlsx(path_to_collection["FULL_PATH"])
collection_df = clean_collection(collection_df)

In [10]:
collection_df

,active,artist,title,record_format,year,genre,price,purchase_date,vinyl_color,lim_edition,number,label,digitized,remarks,rating,artist_country,credit_value,trx_type
28,1.00,Emperor,s/t,MLP,"1,993.00",Black Metal,190.00,2014-08-01,nan,NaN,NaN,NA,False,NaN,nan,None,0,Initial Load
244,1.00,Gatecreeper,Deserted,LP,"2,019.00",Death Metal,70.00,2020-01-01,nan,100,NaN,Relapse Records,True,NaN,8.00,None,0,Initial Load
123,1.00,Agnosy,Traits of The Past,LP,"2,014.00",Crust,15.00,2017-08-01,nan,NaN,NaN,NA,True,NaN,nan,None,0,Initial Load


In [11]:
# Create an additional test_record

test_record = load_some_albums_from_xlsx(path_to_collection["FULL_PATH"], 1, 1)
test_record = clean_collection(test_record)
test_record = test_record.to_dict(orient="records")[0]
test_record["trx_type"] = "Purchase"
test_record["credit_value"] = 1
test_record

{'active': 1.0,
 'artist': 'Witch Vomit',
 'title': 'Poisoned Blood',
 'record_format': 'MLP',
 'year': 2017.0,
 'genre': 'Death Metal',
 'price': 15.0,
 'purchase_date': Timestamp('2020-06-01 00:00:00'),
 'vinyl_color': nan,
 'lim_edition': nan,
 'number': nan,
 'label': '20 Buck Spin',
 'digitized': True,
 'remarks': nan,
 'rating': 8.0,
 'artist_country': None,
 'credit_value': 1,
 'trx_type': 'Purchase'}

### Initial Load of 3 Records

In [12]:
def insert_df_with_sqlalchemy_orm(session, df):
    for x in df.to_dict("records"):
        db_functions.add_new_record(session, x)

In [13]:
insert_df_with_sqlalchemy_orm(session, collection_df)

# assert session.query(Record).count() == 3

C:\Users\r2d4\miniconda3\envs\py3\lib\site-packages\sqlalchemy\sql\sqltypes.py:660: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


### Insertion of 2 Credit Addition Trx

In [14]:
# to_delete = session.query(CreditTrx).filter(CreditTrx.credit_trx_id == 8).one()
# session.delete(to_delete)
# session.commit()

In [15]:
# Initial trx, 11 days ago

addition_trx = CreditTrx(
    credit_trx_date=dt.datetime.today().date() - dt.timedelta(11),
    credit_trx_type="Addition",
    credit_value=1,
    credit_saldo=1,
    record_id=np.nan
)
session.add(addition_trx)

# And a regular interval addition
db_functions.add_regular_credits(session)

session.commit()

Creating 'Addition' Trx for: 2021-02-10


In [16]:
session.query(CreditTrx).all()

[<CreditTrx(credit_trx_id=1, credit_trx_date=2014-08-01, credit_trx_type=Initial Load, credit_value=0.0, credit_saldo=0.0, record_id=1)>,
 <CreditTrx(credit_trx_id=2, credit_trx_date=2020-01-01, credit_trx_type=Initial Load, credit_value=0.0, credit_saldo=0.0, record_id=2)>,
 <CreditTrx(credit_trx_id=3, credit_trx_date=2017-08-01, credit_trx_type=Initial Load, credit_value=0.0, credit_saldo=0.0, record_id=3)>,
 <CreditTrx(credit_trx_id=4, credit_trx_date=2021-01-31, credit_trx_type=Addition, credit_value=1.0, credit_saldo=1.0, record_id=None)>,
 <CreditTrx(credit_trx_id=5, credit_trx_date=2021-02-10, credit_trx_type=Addition, credit_value=1.0, credit_saldo=2.0, record_id=None)>]

### Insertion of new Record

In [17]:
db_functions.add_new_record(session, test_record)

assert session.query(Record).count() == 4

In [18]:
session.query(CreditTrx).all()[-2:]

[<CreditTrx(credit_trx_id=5, credit_trx_date=2021-02-10, credit_trx_type=Addition, credit_value=1.0, credit_saldo=2.0, record_id=None)>,
 <CreditTrx(credit_trx_id=6, credit_trx_date=2020-06-01, credit_trx_type=Purchase, credit_value=-1.0, credit_saldo=1.0, record_id=4)>]

### Removal of Existing Record

Necessary cols: trx_type, credit_value, title, artist, date

In [21]:
test_removal = {
    "trx_type": "Remove",
    "credit_value": 1,
    "artist": "Emperor",
    "title": "s/t",
#     "year": 1993,
    "removal_date": dt.datetime.today().date()
}

In [22]:
db_functions.set_record_to_inactive(session, test_removal)

Status of record 's/t' by Emperor is already 0, please check.


In [23]:
session.query(func.count(distinct(Record.active))).all()
session.query(func.count(Record.record_id)).group_by(Record.active).all()

[(2)]

[(1), (3)]

In [24]:
session.query(CreditTrx).all()[-3:]

[<CreditTrx(credit_trx_id=4, credit_trx_date=2021-01-31, credit_trx_type=Addition, credit_value=1.0, credit_saldo=1.0, record_id=None)>,
 <CreditTrx(credit_trx_id=5, credit_trx_date=2021-02-10, credit_trx_type=Addition, credit_value=1.0, credit_saldo=2.0, record_id=None)>,
 <CreditTrx(credit_trx_id=6, credit_trx_date=2020-06-01, credit_trx_type=Purchase, credit_value=-1.0, credit_saldo=1.0, record_id=4)>]

### Reactivation of inactive Record [OPEN]

In [25]:
# ATTENTION It has to be possible ro re-add inactive records! (and to pay for it in credits!)

## Query DB

In [32]:
pd.read_sql("records", engine)

,record_id,artist_id,title,genre_id,label,year,format_id,vinyl_color,lim_edition,number,remarks,purchase_date,price,digitized,rating,active,created_at,updated_at
0,1,1,s/t,1,None,1993,1,None,None,None,None,2014-08-01,190,0,nan,1,2021-02-11 09:47:44,NaT
1,2,2,Deserted,2,None,2019,2,None,100,None,None,2020-01-01,70,1,8.00,1,2021-02-11 09:47:44,NaT
2,3,3,Traits of The Past,3,None,2014,2,None,None,None,None,2017-08-01,15,1,nan,1,2021-02-11 09:47:44,NaT
3,4,4,Poisoned Blood,2,None,2017,3,None,None,None,None,2020-06-01,15,1,8.00,1,2021-02-11 09:48:13,2021-02-11 10:00:33


In [38]:
pd.read_sql("formats", engine)

,format_id,format_name,created_at,updated_at
0,1,MLP,2021-02-11 09:47:44,NaT
1,2,LP,2021-02-11 09:47:44,NaT
2,3,MLP2,2021-02-11 10:00:33,NaT


In [34]:
pd.read_sql("artists", engine)

,artist_id,artist_name,artist_country,created_at,updated_at
0,1,Emperor,None,2021-02-11 09:47:44,NaT
1,2,Gatecreeper,None,2021-02-11 09:47:44,NaT
2,3,Agnosy,None,2021-02-11 09:47:44,NaT
3,4,Witch Vomit,USA,2021-02-11 09:48:13,2021-02-11 10:00:33


In [35]:
pd.read_sql("labels", engine)

,label_id,label_name,created_at,updated_at
0,1,NA,2021-02-11 09:47:44,NaT
1,2,Relapse Records,2021-02-11 09:47:44,NaT
2,3,20 Buck Spin,2021-02-11 09:48:13,NaT
3,4,Eagle,2021-02-11 10:00:33,NaT


In [36]:
for result in session.query(Record).filter(Record.title == "Poisoned Blood").all():
    print(result.labels)

[<Label(label_id=3, label_name=20 Buck Spin)>, <Label(label_id=4, label_name=Eagle)>]


C:\Users\r2d4\miniconda3\envs\py3\lib\site-packages\sqlalchemy\sql\sqltypes.py:660: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


In [42]:
for result in session.query(Record).filter(Record.format_id == 1).all():
    print(result)

<Record(record_id=1, title=s/t, artist_id=1)>
<Record(record_id=2, title=Deserted, artist_id=2)>


In [ ]:
# TODO: On Updates the old values are not overwritten in many to many relationships (-->Labels) ... new ones are added
# I cound make a correction function that enables updates on many to many entities (labels, genres, artists)

In [30]:
session.close()

# Export

In [46]:
for result in session.query(Record).all():
    print(result)

<Record(record_id=1, title=s/t, artist_id=1)>
<Record(record_id=2, title=Deserted, artist_id=2)>
<Record(record_id=3, title=Traits of The Past, artist_id=3)>
<Record(record_id=4, title=Poisoned Blood, artist_id=4)>


In [92]:
result = session.query(Record).filter(Record.record_id == 4).one_or_none()

# Check
result

<Record(record_id=4, title=Poisoned Blood, artist_id=4)>

In [108]:
# Note for the first export this is still many-to-one, will change to many-to-many in the future

# artist_tuple = (
#     session
#     .query(Artist.artist_name, Artist.artist_country)
#     .join(Record, Record.artist_id == Artist.artist_id)
#     .filter(Artist.artist_id == result.artist_id)
#     .first()
# )            

# # Check
# print(artist_tuple[0])
# print(artist_tuple[1])

result.artist.artist_name
result.artist.artist_country

'Witch Vomit'

'USA'

In [106]:
# One-to-many

# record_format = (
#     session
#     .query(RecordFormat.format_name)
#     .join(Record, Record.format_id == RecordFormat.format_id)
#     .filter(RecordFormat.format_id == result.format_id)
#     .first()[0]
# )            


result.record_format.format_name

MLP2


'MLP2'

In [104]:
# many-to-many

# Check
[label.label_name for label in result.labels]
result.labels[0].label_name

['20 Buck Spin', 'Eagle']

'20 Buck Spin'

In [112]:
record_data_dict = {
    "record_id": result.record_id,
    "artist": result.artist.artist_name,  # has to be adapted to many-to-many
    "artist_country": result.artist.artist_country,  # has to be adapted to many-to-many
    "title": result.title,
    "genre": result.genre.genre_name,
    "label": [label.label_name for label in result.labels],
    "year": result.year,
    "record_format": result.record_format.format_name,
    "vinyl_color": result.vinyl_color,
    "lim_edition": result.lim_edition,
    "number": result.number,
    "remarks": result.remarks,
    "price": result.price,
    "digitized": result.digitized,
    "rating": result.rating, # has to be datapted to one-to-many
    "is_active": result.active,
    "purchase_date": result.purchase_date,
}

In [113]:
record_data_dict

{'record_id': 4,
 'artist': 'Witch Vomit',
 'artist_country': 'USA',
 'title': 'Poisoned Blood',
 'genre': 'Death Metal',
 'label': ['20 Buck Spin', 'Eagle'],
 'year': 2017,
 'record_format': 'MLP2',
 'vinyl_color': 'None',
 'lim_edition': 'None',
 'number': 'None',
 'remarks': 'None',
 'price': Decimal('15.0000000000'),
 'digitized': 1,
 'rating': 8,
 'is_active': 1,
 'purchase_date': datetime.date(2020, 6, 1)}

In [122]:
result_list = session.query(Record).order_by(Record.record_id).all()
dict_list = []
for result in result_list:
    record_data_dict = {
    "record_id": result.record_id,
    "artist": result.artist.artist_name,  # has to be adapted to many-to-many
    "artist_country": result.artist.artist_country,  # has to be adapted to many-to-many
    "title": result.title,
    "genre": result.genre.genre_name,
    "label": [label.label_name for label in result.labels],
    "year": result.year,
    "record_format": result.record_format.format_name,
    "vinyl_color": result.vinyl_color,
    "lim_edition": result.lim_edition,
    "number": result.number,
    "remarks": result.remarks,
    "price": result.price,
    "digitized": result.digitized,
    "rating": result.rating, # has to be datapted to one-to-many
    "is_active": result.active,
    "purchase_date": result.purchase_date,
    }
    dict_list.append(record_data_dict)

records_df = pd.DataFrame(dict_list, columns=dict_list[0].keys())
records_df.set_index('record_id', drop=True, inplace=True) 

assert records_df.index.is_monotonic_increasing & records_df.index.is_unique

In [124]:
records_df.head()

,artist,artist_country,title,genre,label,year,record_format,vinyl_color,lim_edition,number,remarks,price,digitized,rating,is_active,purchase_date
record_id,,,,,,,,,,,,,,,,
1,Emperor,None,s/t,Black Metal,[NA],1993,MLP,None,None,None,None,190.0000000000,0,nan,1,2014-08-01
2,Gatecreeper,None,Deserted,Death Metal,[Relapse Records],2019,MLP,None,100,None,None,70.0000000000,1,8.00,1,2020-01-01
3,Agnosy,None,Traits of The Past,Crust,[NA],2014,LP,None,None,None,None,15.0000000000,1,nan,1,2017-08-01
4,Witch Vomit,USA,Poisoned Blood,Death Metal,"[20 Buck Spin, Eagle]",2017,MLP2,None,None,None,None,15.0000000000,1,8.00,1,2020-06-01


In [127]:
credit_trx_df = pd.read_sql("credit_trx", engine)
credit_trx_df.set_index('credit_trx_id', drop=True, inplace=True) 

assert credit_trx_df.index.is_monotonic_increasing & credit_trx_df.index.is_unique

In [128]:
credit_trx_df.head()

,credit_trx_date,credit_trx_type,credit_value,credit_saldo,record_id,created_at,updated_at
credit_trx_id,,,,,,,
1,2014-08-01,Initial Load,0.00,0.00,1.00,2021-02-11 09:47:44,NaT
2,2020-01-01,Initial Load,0.00,0.00,2.00,2021-02-11 09:47:44,NaT
3,2017-08-01,Initial Load,0.00,0.00,3.00,2021-02-11 09:47:44,NaT
4,2021-01-31,Addition,1.00,1.00,nan,2021-02-11 09:47:47,NaT
5,2021-02-10,Addition,1.00,2.00,nan,2021-02-11 09:47:47,NaT
